In [143]:
import os
import pandas as pd
from unidecode import unidecode

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [144]:
league = {'brasileirao_a': ['Brasileirao'],
          'premier_league': ['Premier_League'],}


liga = league['premier_league'][0]

# IMPORTS E FUNCOES

In [145]:
def remover_acentos(texto):
    return unidecode(texto)

In [146]:
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return pd.NA

In [147]:
historico = pd.read_excel(f'datasets/{liga}/match_history/historico-2015a2023(naotratada).xlsx')
tabela = pd.read_excel(f'datasets/{liga}/standings/tabela-2015a2023(naotratada).xlsx')
rodadas = pd.read_excel(f'datasets/{liga}/rounds/prox_partidas(naotratada).xlsx')

In [148]:
season_atual = historico['season'].max()
rodada_recente = historico[historico['season'] == season_atual]['round'].str.split(' ', expand= True)[1].astype(int).max()

# HISTORICO

### Data cleaning

In [149]:
historico.drop(
  ['notes','captain', 'formation', 'referee', 'match report', 'attendance', 'time'], 
  axis= 'columns', inplace= True)

In [150]:
historico[['gf', 'ga']] = historico[['gf', 'ga']].astype(int)
historico['season'] = historico['season'].astype(int)
historico['round'] = historico['round'].str.split(' ', expand= True)[1].astype(int)

In [151]:
historico['opponent'] = historico['opponent'].apply(remover_acentos)
historico['opponent'] = [linha.lower() for linha in historico['opponent']]
historico['opponent'] = historico['opponent'].str.replace(' ', '_')
historico['opponent'] = historico['opponent'].str.replace('(', '')
historico['opponent'] = historico['opponent'].str.replace(')', '')
historico['opponent'] = historico['opponent'].str.replace(')', '')

# Especifico para o brasileirão
historico['opponent'] = historico['opponent'].str.replace('atletico', 'atl')
historico['team'] = historico['team'].str.replace('athletico', 'ath')
historico['team'] = historico['team'].str.replace('atletico', 'atl')

# Especifico para a Premier League
historico['team'] = historico['team'].str.replace('_hotspur', '')
historico['team'] = historico['team'].str.replace('rhampton_wanderers', 's')
historico['team'] = historico['team'].str.replace('united', 'utd')
historico['team'] = historico['team'].str.replace('west_ham_utd', 'west_ham')
historico['team'] = historico['team'].str.replace('_and_hove_albion', '')
historico['team'] = historico['team'].str.replace('wich_albion', '')
historico['team'] = historico['team'].str.replace('_town', '')
historico['team'] = historico['team'].str.replace('luton', 'luton_town')
historico['opponent'] = historico['opponent'].str.replace('united', 'utd')
historico['opponent'] = historico['opponent'].str.replace("nott'ham_forest", "nottingham_forest")

In [152]:
[item for item in historico['opponent'].unique() if item not in historico['team'].unique()]

[]

In [153]:
historico['team'].unique()

array(['tottenham', 'manchester_city', 'arsenal', 'liverpool',
       'aston_villa', 'newcastle_utd', 'brighton', 'manchester_utd',
       'west_ham', 'chelsea', 'crystal_palace', 'wolves', 'fulham',
       'brentford', 'nottingham_forest', 'everton', 'luton_town',
       'burnley', 'bournemouth', 'sheffield_utd', 'leicester_city',
       'leeds_utd', 'southampton', 'watford', 'norwich_city', 'west_brom',
       'cardiff_city', 'huddersfield', 'swansea_city', 'stoke_city',
       'hull_city', 'middlesbrough', 'sunderland'], dtype=object)

In [154]:
historico.head()

,date,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,sh,sot,season,team
0,2023-08-13,Premier League,1,Sun,Away,D,2,2,brentford,1.3,2.2,69,18,6,2023,tottenham
1,2023-08-19,Premier League,2,Sat,Home,W,2,0,manchester_utd,1.7,2.1,55,17,6,2023,tottenham
2,2023-08-26,Premier League,3,Sat,Away,W,2,0,bournemouth,2.1,0.7,56,17,6,2023,tottenham
3,2023-09-02,Premier League,4,Sat,Away,W,5,2,burnley,2.2,1.3,53,21,11,2023,tottenham
4,2023-09-16,Premier League,5,Sat,Home,W,2,1,sheffield_utd,2.1,0.7,70,28,10,2023,tottenham


### Divisão e Export

Todos

In [155]:
historico.to_excel(f'datasets/{liga}/match_history/all_games/historico-2015a2023.xlsx', index=False)

In [156]:
for season in historico['season'].unique():
  if not os.path.exists(f'datasets/{liga}/match_history/all_games/season-{season}'):
    os.makedirs(f'datasets/{liga}/match_history/all_games/season-{season}')
  df = historico[historico['season'] == season]
  df.to_excel(f'datasets/{liga}/match_history/all_games/season-{season}/historico.xlsx', index= False)

Casa

In [157]:
historico_casa = historico[historico['venue'] == 'Home']
historico_casa.to_excel(f'datasets/{liga}/match_history/home_only/historico-2015a2023.xlsx', index=False)

In [158]:
for season in historico_casa['season'].unique():
  if not os.path.exists(f'datasets/{liga}/match_history/home_only/season-{season}'):
    os.makedirs(f'datasets/{liga}/match_history/home_only/season-{season}')
  df = historico_casa[historico_casa['season'] == season]
  df.to_excel(f'datasets/{liga}/match_history/home_only/season-{season}/historico.xlsx', index= False)

Por Time

In [159]:
for time in historico['team'].unique():
  df = historico[historico['team'] == time]
  df.to_excel(f'datasets/{liga}/match_history/teams/{time}.xlsx', index= False)

# TABELA

### Data Cleaning

In [160]:
tabela.drop(
  ['attendance', 'top team scorer', 'goalkeeper', 'notes', 'last 5'], 
  axis= 'columns', inplace= True)

In [161]:
tabela['squad'] = tabela['squad'].apply(remover_acentos)
tabela['squad'] = [linha.lower() for linha in tabela['squad']]
tabela['squad'] = tabela['squad'].str.replace(' ', '_')
tabela['squad'] = tabela['squad'].str.replace('(', '')
tabela['squad'] = tabela['squad'].str.replace(')', '')
tabela['squad'] = tabela['squad'].str.replace(')', '')

# Especifico para o brasileirão
tabela['squad'] = tabela['squad'].str.replace('atletico', 'atl')

# Especifico para a Premier League
tabela['squad'] = tabela['squad'].str.replace("united", "utd")
tabela['squad'] = tabela['squad'].str.replace("nott'ham_forest", "nottingham_forest")

In [162]:
[item for item in tabela['squad'].unique() if item not in historico['opponent'].unique()]

[]

In [163]:
historico['opponent'].unique()

array(['brentford', 'manchester_utd', 'bournemouth', 'burnley',
       'sheffield_utd', 'arsenal', 'liverpool', 'luton_town', 'fulham',
       'crystal_palace', 'newcastle_utd', 'west_ham', 'nottingham_forest',
       'wolves', 'brighton', 'everton', 'tottenham', 'manchester_city',
       'chelsea', 'aston_villa', 'southampton', 'leicester_city',
       'leeds_utd', 'norwich_city', 'watford', 'west_brom',
       'huddersfield', 'cardiff_city', 'stoke_city', 'swansea_city',
       'hull_city', 'middlesbrough', 'sunderland'], dtype=object)

### Feature Engineering

In [164]:
tabela['xg_conv'] = tabela['gf'] - tabela['xg']      #Conversão de xG
tabela['xga_conv'] = tabela['ga'] - tabela['xga']    #Conversão de xGA
tabela['att_rating'] = (tabela['gf']/tabela['mp'])/(tabela['gf']/tabela['mp']).mean()  #Media de gols por partida dividido pelo da liga
tabela['def_rating'] = (tabela['ga']/tabela['mp'])/(tabela['gf']/tabela['mp']).mean()  #Media de gols tomados por partida dividido pelo da liga
tabela['naive_rating'] = tabela['att_rating']*tabela['def_rating']*(tabela['gf']/tabela['mp']).mean()   #att*def*media da liga

for time in tabela['squad']:
  #xG
  tabela.loc[tabela['squad'] == time, 'xg_casa'] = historico_casa[historico_casa['team'] == time]['xg'].mean()        #xG em casa
  tabela.loc[tabela['squad'] == time, 'xg_fora'] = historico_casa[historico_casa['opponent'] == time]['xga'].mean()   #xG fora
  tabela.loc[tabela['squad'] == time, 'xg_total'] = (tabela['xg_casa'] + tabela['xg_fora'])/2                         #xG total

  #xGA
  tabela.loc[tabela['squad'] == time, 'xga_casa'] = historico_casa[historico_casa['team'] == time]['xga'].mean()      #xGA em casa
  tabela.loc[tabela['squad'] == time, 'xga_fora'] = historico_casa[historico_casa['opponent'] == time]['xg'].mean()   #xGA fora
  tabela.loc[tabela['squad'] == time, 'xga_total'] = (tabela['xga_casa'] + tabela['xga_fora'])/2                      #xGA total

  #GF
  tabela.loc[tabela['squad'] == time, 'gf_casa'] = historico_casa[historico_casa['team'] == time]['gf'].mean()        #GF em casa
  tabela.loc[tabela['squad'] == time, 'gf_fora'] = historico_casa[historico_casa['opponent'] == time]['ga'].mean()    #GF fora
  tabela.loc[tabela['squad'] == time, 'gf_total'] = tabela['gf_casa'] + tabela['gf_fora']                             #GF total

  #GA
  tabela.loc[tabela['squad'] == time, 'ga_casa'] = historico_casa[historico_casa['team'] == time]['ga'].mean()        #GA em casa
  tabela.loc[tabela['squad'] == time, 'ga_fora'] = historico_casa[historico_casa['opponent'] == time]['gf'].mean()    #GA fora
  tabela.loc[tabela['squad'] == time, 'ga_total'] = tabela['ga_casa'] + tabela['ga_fora']                             #GA total


In [165]:
#Ultimos 5 jogos
historico_5jogos = historico_casa[historico_casa['round'] > rodada_recente-5]

for time in tabela['squad']:
  #xG
  tabela.loc[tabela['squad'] == time, 'xg_5casa'] = historico_5jogos[historico_5jogos['team'] == time]['xg'].mean()       #xG em casa
  tabela.loc[tabela['squad'] == time, 'xg_5fora'] = historico_5jogos[historico_5jogos['opponent'] == time]['xga'].mean()  #xG fora
  tabela.loc[tabela['squad'] == time, 'xg_5total'] = (tabela['xg_5casa'] + tabela['xg_5fora'])/2                          #xG total

  #xGA
  tabela.loc[tabela['squad'] == time, 'xga_5casa'] = historico_5jogos[historico_5jogos['team'] == time]['xga'].mean()     #xGA em casa
  tabela.loc[tabela['squad'] == time, 'xga_5fora'] = historico_5jogos[historico_5jogos['opponent'] == time]['xg'].mean()  #xGA fora
  tabela.loc[tabela['squad'] == time, 'xga_5total'] = (tabela['xga_5casa'] + tabela['xga_5fora'])/2                       #xGA total

  #GF
  tabela.loc[tabela['squad'] == time, 'gf_5casa'] = historico_5jogos[historico_5jogos['team'] == time]['gf'].mean()       #GF em casa
  tabela.loc[tabela['squad'] == time, 'gf_5fora'] = historico_5jogos[historico_5jogos['opponent'] == time]['ga'].mean()   #GF fora
  tabela.loc[tabela['squad'] == time, 'gf_5total'] = tabela['gf_5casa'] + tabela['gf_5fora']                              #GF total

  #GA
  tabela.loc[tabela['squad'] == time, 'ga_5casa'] = historico_5jogos[historico_5jogos['team'] == time]['ga'].mean()       #GA em casa
  tabela.loc[tabela['squad'] == time, 'ga_5fora'] = historico_5jogos[historico_5jogos['opponent'] == time]['gf'].mean()   #GA fora
  tabela.loc[tabela['squad'] == time, 'ga_5total'] = tabela['ga_5casa'] + tabela['ga_5fora']                              #GA total
  

### Divisões e Exports

In [166]:
tabela_atual = tabela[tabela['season'] == 2023]

In [167]:
for season in tabela['season'].unique()[1:]:
  if not os.path.exists(f'datasets/{liga}/standings/all_standings/season-{season}'):
    os.makedirs(f'datasets/{liga}/standings/all_standings/season-{season}')
  df = tabela[tabela['season'] == season]
  df.to_excel(f'datasets/{liga}/standings/all_standings/season-{season}/tabela.xlsx', index= False)


if not os.path.exists(f'datasets/{liga}/standings/all_standings/season-{season_atual}'):
  os.makedirs(f'datasets/{liga}/standings/all_standings/season-{season_atual}')
tabela_atual.to_excel(f'datasets/{liga}/standings/all_standings/season-{season_atual}/tabela-rodada{rodada_recente}.xlsx', index= False)

### Tabelas casa e fora

In [168]:
historico_atual = historico[historico['season'] == season_atual]
tabela_casa = []
tabela_fora = []
for time in historico_atual['team'].unique():
  tabela_casa.append(sum(historico_atual[historico_atual['venue'] == 'Home'].loc[historico_atual['team'] == time]['result'].map({'W': 3, 'D': 1, 'L': 0})))
  tabela_fora.append(sum(historico_atual[historico_atual['venue'] == 'Away'].loc[historico_atual['team'] == time]['result'].map({'W': 3, 'D': 1, 'L': 0})))

In [169]:
tabela_atual['pts_casa'] = tabela_casa
tabela_atual['pts_fora'] = tabela_fora

In [170]:
tabela_fora = tabela_atual.sort_values(by= 'pts_fora', ascending= False, ignore_index= True)
tabela_fora['rk'] = range(1,21)

In [171]:
tabela_casa = tabela_atual.sort_values(by= 'pts_casa', ascending= False, ignore_index= True)
tabela_casa['rk'] = range(1,21)

In [172]:
if not os.path.exists(f'datasets/{liga}/standings/home_standings/season-{season_atual}'):
  os.makedirs(f'datasets/{liga}/standings/home_standings/season-{season_atual}')
if not os.path.exists(f'datasets/{liga}/standings/away_standings/season-{season_atual}'):
  os.makedirs(f'datasets/{liga}/standings/away_standings/season-{season_atual}')

tabela_casa.to_excel(f'datasets/{liga}/standings/home_standings/season-{season_atual}/tabela-rodada{rodada_recente}.xlsx', index= False)
tabela_fora.to_excel(f'datasets/{liga}/standings/away_standings/season-{season_atual}/tabela-rodada{rodada_recente}.xlsx', index= False)

# RODADAS

### Data Cleaning

In [173]:
rodadas = rodadas.loc[rodadas['home'].notna()]

In [174]:
rodadas.drop(['day', 'time', 'attendance', 'venue', 'referee', 'match report', 'notes'], axis= 1, inplace= True)

In [175]:
rodadas['home'] = rodadas['home'].apply(remover_acentos)
rodadas['home'] = [linha.lower() for linha in rodadas['home']]
rodadas['home'] = rodadas['home'].str.replace(' ', '_')
rodadas['home'] = rodadas['home'].str.replace('(', '')
rodadas['home'] = rodadas['home'].str.replace(')', '')
rodadas['home'] = rodadas['home'].str.replace(')', '')

# Especifico para o brasileirão
rodadas['home'] = rodadas['home'].str.replace('atletico', 'atl')

# Especifico para a Premier League
rodadas['home'] = rodadas['home'].str.replace("nott'ham_forest", "nottingham_forest")

In [176]:
rodadas['away'] = rodadas['away'].apply(remover_acentos)
rodadas['away'] = [linha.lower() for linha in rodadas['away']]
rodadas['away'] = rodadas['away'].str.replace(' ', '_')
rodadas['away'] = rodadas['away'].str.replace('(', '')
rodadas['away'] = rodadas['away'].str.replace(')', '')
rodadas['away'] = rodadas['away'].str.replace(')', '')

# Especifico para o brasileirão
rodadas['away'] = rodadas['away'].str.replace('atletico', 'atl')

# Especifico para a Premier League
rodadas['away'] = rodadas['away'].str.replace("nott'ham_forest", "nottingham_forest")

In [177]:
[item for item in rodadas['home'].unique() if item not in historico['opponent'].unique()]

[]

In [178]:
rodadas[['gols_casa', 'gols_fora']] = rodadas['score'].str.split('–', expand=True)
rodadas.drop(['score'], axis= 1, inplace= True)
rodadas.insert(5, 'gols_casa', rodadas.pop('gols_casa'))
rodadas.insert(6, 'gols_fora', rodadas.pop('gols_fora'))

rodadas = rodadas.rename(columns={'xg.1': 'xg_fora', 'xg': 'xg_casa'})

rodadas['gols_casa'] = rodadas['gols_casa'].apply(convert_to_int)
rodadas['gols_fora'] = rodadas['gols_fora'].apply(convert_to_int)


In [179]:
rodadas.head()

,wk,date,home,xg_casa,xg_fora,gols_casa,gols_fora,away
0,1.0,2023-08-11,burnley,0.3,1.9,0,3,manchester_city
1,1.0,2023-08-12,arsenal,0.8,1.2,2,1,nottingham_forest
2,1.0,2023-08-12,everton,2.7,1.5,0,1,fulham
3,1.0,2023-08-12,sheffield_utd,0.5,1.9,0,1,crystal_palace
4,1.0,2023-08-12,brighton,4.0,1.5,4,1,luton_town


### Divisão e Export

In [180]:
rodadas.to_excel(f'datasets/Brasileirao/rounds/all_rounds/rodadas.xlsx', index= False)